In [14]:
import numpy as np
import pandas as pd

In [15]:
data = pd.read_csv('/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/caption/gemini_labeled_28k.csv')

In [16]:
data["candidates"] = data["candidates"].apply(lambda x: list(set(eval(x))))
data["hallucination_candidates"] = data["hallucination_candidates"].apply(lambda x: list(set(eval(x))))

In [17]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates
0,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"[bat, one, holding, back, tennis, fencing, sid...",[bat]
1,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"[bat, one, holding, back, tennis, frisbee, fen...","[bat, frisbee]"


In [20]:
import re
import unicodedata
from typing import Dict, Iterable, List, Tuple

def find_word_spans(
    text: str,
    terms: Iterable[str],
    *,
    case_sensitive: bool = False,
    whole_word: bool = False,
    overlapping: bool = False,
    normalize_unicode: bool = True,
) -> Dict[str, List[Tuple[int, int]]]:
    
    if normalize_unicode:
        text = unicodedata.normalize("NFC", text)

    # Prepare result with original (possibly unnormalized) term strings as keys
    results: Dict[str, List[Tuple[int, int]]] = {t: [] for t in terms if t}

    # Build a working list of (orig_term, norm_term)
    work_terms: List[Tuple[str, str]] = []
    for t in terms:
        if not t:
            continue
        t_norm = unicodedata.normalize("NFC", t) if normalize_unicode else t
        work_terms.append((t, t_norm))

    flags = 0 if case_sensitive else re.IGNORECASE

    for orig_term, norm_term in work_terms:
        # Escape to match literal text
        escaped = re.escape(norm_term)

        if whole_word:
            # Word boundaries only at the ends (so phrases are supported)
            # Use (?<!\w) and (?!\w) instead of \b to behave well inside lookaheads for overlapping.
            core = f"(?<!\\w){escaped}(?!\\w)"
        else:
            core = escaped

        if overlapping:
            # Lookahead to capture overlapping starts.
            # Put the core inside a capturing group so we can compute the end index.
            pattern = re.compile(f"(?=({core}))", flags)
            matches = pattern.finditer(text)
            for m in matches:
                start = m.start()
                end = start + len(m.group(1))
                results[orig_term].append((start, end))
        else:
            pattern = re.compile(core, flags)
            matches = pattern.finditer(text)
            for m in matches:
                results[orig_term].append((m.start(), m.end()))

    return results


In [18]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates
0,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"[bat, one, holding, back, tennis, fencing, sid...",[bat]
1,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"[bat, one, holding, back, tennis, frisbee, fen...","[bat, frisbee]"


In [30]:
find_word_spans(
    text=data.iloc[100]["answer"].lower(),
    terms=[i.lower() for i in data.iloc[100]["candidates"]])

{'vessels': [(58, 65)],
 'pink': [(193, 197), (234, 238)],
 'light': [(548, 553)],
 'side': [(506, 510)],
 'pillows': [(122, 129), (182, 189)],
 'background': [(303, 313)],
 'frames': [(331, 337), (528, 534)],
 'carpet': [(137, 143)],
 'right': [(500, 505)],
 'arranged': [(16, 24)],
 'cups': [(30, 34)],
 'buildings': [(482, 491)],
 'brown': [(100, 105), (389, 394)],
 'white': [(436, 441)],
 'steel': [(52, 57)],
 'table': [(91, 96)],
 'two': [(26, 29)],
 'plants': [(474, 480)],
 'sofa': [(226, 230)],
 'books': [(10, 15)],
 'objects': [(76, 83), (564, 571)],
 'grey': [(208, 212), (253, 257), (284, 288), (359, 363)],
 'orange': [(38, 44)],
 'curtain': [(425, 432)],
 'floor': [(160, 165)],
 'wall': [(345, 349), (542, 546)],
 'window': [(409, 415)],
 'photo': [(325, 330), (522, 527)],
 'roof': [(375, 379)],
 'chair': [(269, 274)]}